<a href="https://colab.research.google.com/github/LuisaPolicarpo/Project3/blob/main/Project3_Recomendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive

/content/drive/MyDrive


In [4]:
#Import table
df_inovmovie = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/Copy_movies_imdb.pickle')
df_inovmovie

,primaryName,tconst,genres,primaryTitle,averageRating,numVotes,primaryProfession,titleType,startYear,wheighted_IMDB
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,5.7,1922,cinematographer,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Short,Carmencita,5.7,1922,cinematographer,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,5.7,1922,director,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Short,Carmencita,5.7,1922,director,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,5.7,1922,producer,short,1894,6.278611
...,...,...,...,...,...,...,...,...,...,...
3444278,Oliver Bickers,tt9916544,Drama,My Sweet Prince,6.9,60,actor,short,2019,6.324476
3444278,Oliver Bickers,tt9916544,Short,My Sweet Prince,6.9,60,actor,short,2019,6.324476
3444279,Ganesh Vasant Patil,tt9916730,\N,6 Gunn,9.2,7,actor,movie,2017,6.323900
3444281,Ujjwala Gawde,tt9916730,\N,6 Gunn,9.2,7,producer,movie,2017,6.323900


In [5]:
df_inovmovie['startYear'].unique()

array(['1894', '1893', '1896', '1899', '1903', '1897', '1895', '1898',
       '1892', '1891', '2018', '1890', '1901', '1878', '1877', '1979',
       '1965', '1910', '1914', '1942', '1940', '1946', '1913', '1922',
       '1911', '2019', '2015', '1904', '1920', '1933', '1929', '1905',
       '1900', '1902', '1945', '1996', '1966', '1874', '1909', '1907',
       '1908', '1906', '1912', '1939', '1978', '1921', '1915', '1919',
       '1918', '1923', '1924', '1925', '1928', '1951', '1934', '1917',
       '1926', '1927', '1930', '1931', '1932', '1935', '1936', '1937',
       '1938', '1941', '1944', '1947', '1949', '1955', '1956', '1952',
       '1991', '1916', '2003', '2004', '1992', '1948', '1983', '1950',
       '2020', '1953', '1959', '1960', '1964', '1967', '1972', '1974',
       '1976', '1977', '1984', '2002', '2005', '1982', '2006', '1980',
       '2021', '2012', '2013', '2014', '2007', '1969', '1990', '2009',
       '2022', '2011', '2016', '2017', '1943', '1954', '1957', '1958',
      

In [6]:
condition = df_inovmovie.loc[df_inovmovie['startYear'] != '\\N']


In [7]:
#drop averageRating, numVotes
condition.drop(['averageRating', 'numVotes'], axis = 1, inplace = True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
#Genre get dummies

condition = pd.concat([condition , condition['genres'].str.get_dummies()], 
          axis = 1)

In [9]:
condition

,primaryName,tconst,genres,primaryTitle,primaryProfession,titleType,startYear,wheighted_IMDB,Action,Adult,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,cinematographer,short,1894,6.278611,0,0,...,0,0,0,0,0,0,0,0,0,0
1,William K.L. Dickson,tt0000001,Short,Carmencita,cinematographer,short,1894,6.278611,0,0,...,0,0,0,1,0,0,0,0,0,0
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,director,short,1894,6.278611,0,0,...,0,0,0,0,0,0,0,0,0,0
1,William K.L. Dickson,tt0000001,Short,Carmencita,director,short,1894,6.278611,0,0,...,0,0,0,1,0,0,0,0,0,0
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,producer,short,1894,6.278611,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444278,Oliver Bickers,tt9916544,Drama,My Sweet Prince,actor,short,2019,6.324476,0,0,...,0,0,0,0,0,0,0,0,0,0
3444278,Oliver Bickers,tt9916544,Short,My Sweet Prince,actor,short,2019,6.324476,0,0,...,0,0,0,1,0,0,0,0,0,0
3444279,Ganesh Vasant Patil,tt9916730,\N,6 Gunn,actor,movie,2017,6.323900,0,0,...,0,0,0,0,0,0,0,0,0,1
3444281,Ujjwala Gawde,tt9916730,\N,6 Gunn,producer,movie,2017,6.323900,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
condition_g = pd.pivot_table(data = condition, values = condition[['primaryTitle', 'titleType', 'startYear', 'wheighted_IMDB', 'Action', 'Adult',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War',
       'Western']], index = condition['tconst'], aggfunc='mean')
condition_g


,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,wheighted_IMDB
tconst,,,,,,,,,,,,,,,,,,,,,
tt0000001,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.5,0.0,0.0,...,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,6.278611
tt0000002,0.0,0.0,0.000000,0.500000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,6.317731
tt0000003,0.0,0.0,0.000000,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,6.334569
tt0000004,0.0,0.0,0.000000,0.500000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,6.318096
tt0000005,0.0,0.0,0.000000,0.000000,0.0,0.500000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,6.311721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9916428,0.0,0.0,0.333333,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,6.321682
tt9916538,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,6.323569
tt9916544,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.5,0.0,...,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,6.324476


In [11]:
#Clean the nans

# condition = df_inovmovie['Crime'] == r'\\N'
# condition.apply(lambda x : '1' if x == 1 else 0).sum()

In [12]:
# df_inovmovie['startYear'].str.contains(r'\\N', regex=True).sum()

In [13]:
# df_inovmovie['startYear_n'] = df_inovmovie['startYear'].apply(lambda x : np.nan if x == r'\\N' else x)


In [14]:
# df_inovmovie.dropna(subset=['startYear_n'], inplace = True)
# # df_inovmovie['startYear_n'].isna().sum()

In [15]:
# df_inovmovie['startYear_n'].isna().sum()

In [16]:
#convert date type
# import datetime

# df_inovmovie['startYear'] = pd.to_datetime(df_inovmovie['startYear'], format='%Y')

condition['startYear'].astype('int')


1          1894
1          1894
1          1894
1          1894
1          1894
           ... 
3444278    2019
3444278    2019
3444279    2017
3444281    2017
3444282    2017
Name: startYear, Length: 7494710, dtype: int64

In [17]:
#NList of columns to choose from for the model
condition.columns

Index(['primaryName', 'tconst', 'genres', 'primaryTitle', 'primaryProfession',
       'titleType', 'startYear', 'wheighted_IMDB', 'Action', 'Adult',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War',
       'Western', '\N'],
      dtype='object')

In [18]:
#KNN model

X = condition[['startYear', 'wheighted_IMDB',
       'Adventure']]

model = NearestNeighbors(n_neighbors=5).fit(X)

In [19]:
name = ['Carmencita']

array1, array2 = model.kneighbors(condition.loc[condition['primaryName'].isin(name), ['startYear', 'wheighted_IMDB', 'Adventure']])
array1, array2

(array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[2256532, 1797585, 2256531, 2256533, 1797582],
        [2256532, 1797585, 2256531, 2256533, 1797582]]))

In [20]:
list_1 = array1.tolist()
list_2 = array2.tolist()

flat_list1 = list(np.concatenate(list_1).flat)
flat_list2 = list(np.concatenate(list_2).flat)

d = {'Distance': flat_list1,'#': flat_list2}
df12 = pd.DataFrame(d)

dfnl_df12 = pd.merge(df_notLegendary, df12, how='inner')
dfnl_df12.sort_values(by = 'Distance').head(5)

NameError: ignored